In [1]:
import json
import re
import os
import time
from typing import Dict, Any, List
from tqdm import tqdm
import litellm
from openai import OpenAI, AsyncOpenAI
from datasets import load_dataset
import asyncio
import random

/home/vashistt/miniforge3/envs/verl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Graph# add task, unique_combined_id: graph_id
# {'id': 791,
#  'graph_params': {'N': 10, 'M': 8, 'W': 111, 'P': 1},
#  'edges': [[0, 1, 51],
#   [0, 2, 72],
#   [0, 3, 97],
#   [0, 4, 37],
#   [0, 5, 32],
#   [0, 6, 107],
#   [0, 7, 17],
#   [0, 8, 3],
#   [1, 2, 96],
#   [1, 3, 18],
#   [1, 4, 16],
#   [1, 5, 82],
#   [1, 6, 66],
#   [1, 7, 1],
#   [1, 8, 45],
#   [1, 9, 39],
#   [2, 3, 35],
#   [2, 4, 79],
#   [2, 5, 98],
#   [2, 6, 71],
#   [2, 7, 64],
#   [2, 8, 3],
#   [2, 9, 55],
# ...
#   [9, 5, 60],
#   [9, 6, 23],
#   [9, 7, 81]],
#  'prompt': 'You are given a directed graph with 10 nodes (numbered 0 to 9) and the following edges:\n\nEdges (source -> target, weight):\n0 -> 1, weight: 51\n0 -> 2, weight: 72\n0 -> 3, weight: 97\n0 -> 4, weight: 37\n0 -> 5, weight: 32\n0 -> 6, weight: 107\n0 -> 7, weight: 17\n0 -> 8, weight: 3\n1 -> 2, weight: 96\n1 -> 3, weight: 18\n1 -> 4, weight: 16\n1 -> 5, weight: 82\n1 -> 6, weight: 66\n1 -> 7, weight: 1\n1 -> 8, weight: 45\n1 -> 9, weight: 39\n2 -> 3, weight: 35\n2 -> 4, weight: 79\n2 -> 5, weight: 98\n2 -> 6, weight: 71\n2 -> 7, weight: 64\n2 -> 8, weight: 3\n2 -> 9, weight: 55\n2 -> 0, weight: 38\n3 -> 4, weight: 60\n3 -> 5, weight: 77\n3 -> 6, weight: 96\n3 -> 7, weight: 9\n3 -> 8, weight: 52\n3 -> 9, weight: 72\n3 -> 0, weight: 3\n3 -> 1, weight: 6\n4 -> 5, weight: 57\n4 -> 6, weight: 8\n4 -> 7, weight: 76\n4 -> 8, weight: 20\n4 -> 9, weight: 102\n4 -> 0, weight: 10\n4 -> 1, weight: 79\n4 -> 2, weight: 72\n5 -> 6, weight: 9\n5 -> 7, weight: 99\n5 -> 8, weight: 95\n5 -> 9, weight: 52\n5 -> 0, weight: 71\n5 -> 1, weight: 98\n5 -> 2, weight: 59\n5 -> 3, weight: 45\n6 -> 7, weight: 53\n6 -> 8, weight: 105\n6 -> 9, weight: 98\n6 -> 0, weight: 13\n6 -> 1, weight: 32\n6 -> 2, weight: 105\n6 -> 3, weight: 22\n6 -> 4, weight: 11\n7 -> 8, weight: 32\n7 -> 9, weight: 25\n7 -> 0, weight: 68\n7 -> 1, weight: 106\n7 -> 2, weight: 1\n7 -> 3, weight: 111\n7 -> 4, weight: 94\n7 -> 5, weight: 84\n8 -> 9, weight: 22\n8 -> 0, weight: 15\n8 -> 1, weight: 72\n8 -> 2, weight: 18\n8 -> 3, weight: 68\n8 -> 4, weight: 57\n8 -> 5, weight: 81\n8 -> 6, weight: 49\n9 -> 0, weight: 44\n9 -> 1, weight: 103\n9 -> 2, weight: 110\n9 -> 3, weight: 25\n9 -> 4, weight: 45\n9 -> 5, weight: 60\n9 -> 6, weight: 23\n9 -> 7, weight: 81\n\nFind the top 1 shortest path from node 0 to node 9.\n\nReturn your answer by calling the submit_paths function with:\n- paths: A list of paths, where each path is a list of node indices\n- weights: A list of corresponding path weights\n\nFor example, if the shortest path is [0, 2, 4] with weight 10, call:\nsubmit_paths(paths=[[0, 2, 4]], weights=[10])\n',
#  'solution': {'paths': [{'path': [0, 8, 9], 'weight': 25}]}}

# # infobench
# {'id': 'domain_oriented_task_186',
#  'input': '',
#  'category': 'Engineering and Technology: Artificial Intelligence',
#  'instruction': 'Generate a Python code snippet that constructs a two-hidden layer feedforward neural network using the PyTorch `torch.nn` module, ensuring that the architecture consists of an input layer, two hidden layers, and an output layer. The hidden layers should utilize the ReLU activation function and have a neuron count ranging between 32 and 128 to maintain a reasonably sized network. Include comments in the code to clarify each step of the process.',
#  'decomposed_questions': ['Is the generated text a Python code snippet?',
#   'Does the generated Python code construct a two-hidden layer feedforward neural network using the PyTorch `torch.nn` module?',
#   'Does the architecture designed in the Python code consist of an input layer, two hidden layers, and an output layer?',
#   'Does the Python code ensure that the hidden layers of the network utilize the ReLU activation function?',
#   'Does the neuron count in each hidden layer of the network, as implemented in the Python code, range between 32 and 128 to maintain a reasonably sized network?',
#   'Does the generated Python code include comments that clarify each step of the process?'],
#  'subset': 'Hard_set',
#  'question_label': [['Format', 'Linguistic'],
#   ['Content'],
#   ['Content'],
#   ['Content'],
#   ['Content'],
#   ['Format']]}

# # MMLU_med
# {'question': 'Which of the following releases most energy when completely oxidised in the body?',
#  'subject': 'college_medicine',
#  'choices': ['One gram of glucose',
#   'One gram of palmitic acid',
#   'One gram of leucine',
#   'One gram of alcohol'],
#  'answer': 1}
# # def get_ground_truth(self, example: Dict[str, Any]) -> Any:
# #         answer_int = example["answer"]
# #         answer_letter = chr(65 + answer_int)
# #         return answer_letter

In [3]:
def process_graph_entry(ex, idx: int):
    """
    Process Graph: Uses the pre-written prompt field which describes the graph.
    index: a 1-based integer provided by the caller
    """
    return {
        "index": idx,  # integer index (1..n_examples for this dataset)
        "task": "graph",
        "prompt": ex.get('prompt'),
        "gold_answer": ex.get('solution'),
        "meta": {
            "graph_params": ex.get('graph_params'),
            "edges": ex.get('edges'),
            "original_id": ex.get('id')  # keep original unique id here
        }
    }

In [4]:
def process_infobench_entry(ex, idx: int):
    """
    Process Infobench: Uses 'instruction'.
    If 'input' exists (context), it appends it.
    """
    raw_instruction = ex.get('instruction', '') or ''
    context = ex.get('input', '') or ''

    if context:
        full_prompt = f"{raw_instruction}\n\n{context}"
    else:
        full_prompt = raw_instruction

    return {
        "index": idx,
        "task": "infobench",
        "prompt": full_prompt,
        "gold_answer": None,
        "meta": {
            "decomposed_questions": ex.get('decomposed_questions'),
            "category": ex.get('category'),
            "subset": ex.get('subset'),
            "original_id": ex.get('id')
        }
    }

In [5]:
def process_mmlu_med_entry(ex, idx: int):
    """
    Process MMLU: Formats Question + Choices (A, B, C, D).
    Converts integer answer (0) to Letter (A).
    """
    q = ex.get("question", "") or ""
    choices = ex.get("choices", []) or []

    formatted_choices = "\n".join([f"{chr(65+i)}. {c}" for i, c in enumerate(choices)])

    prompt_text = f"Question:\n{q}\n\nChoices:\n{formatted_choices}"

    answer_idx = ex.get("answer")
    gold_letter = chr(65 + answer_idx) if answer_idx is not None else None

    return {
        "index": idx,
        "task": "mmlu_med",
        "prompt": prompt_text,
        "gold_answer": gold_letter,
        "meta": {
            "subject": ex.get("subject"),
            "answer_index": answer_idx,
            "raw_choices": choices,
            "original_id": ex.get('id')
        }
    }


In [6]:
def get_combined_dataset(n_each=5):
    combined_data = []

    # Initialize a global index counter starting at 1
    global_idx = 1

    print("Loading datasets...")
    graph_ds = load_dataset("vashistht/11763_datasets", 'graph_dev', split='dev_test')
    infobench_ds = load_dataset("vashistht/11763_datasets", 'infobench', split='dev_test')
    mmlu_ds = load_dataset("vashistht/11763_datasets", 'mmlu_med', split='dev_test')

    # --- Graph ---
    print(f"Processing {n_each} examples from Graph...")
    for ex in graph_ds.select(range(n_each)):
        # Pass the current global_idx
        combined_data.append(process_graph_entry(ex, global_idx))
        global_idx += 1  # Increment for the next item

    # --- Infobench ---
    print(f"Processing {n_each} examples from Infobench...")
    for ex in infobench_ds.select(range(n_each)):
        # global_idx continues from where Graph left off
        combined_data.append(process_infobench_entry(ex, global_idx))
        global_idx += 1

    # --- MMLU Med ---
    print(f"Processing {n_each} examples from MMLU Med...")
    for ex in mmlu_ds.select(range(n_each)):
        combined_data.append(process_mmlu_med_entry(ex, global_idx))
        global_idx += 1

    # Shuffle the combined data
    # Note: The indices will now be unique (1..N), but scrambled in order.
    random.shuffle(combined_data)

    return combined_data

In [7]:
dataset = get_combined_dataset(n_each=2)

def print_sample(task_name, data):
    item = next((x for x in data if x['task'] == task_name), None)
    if item:
        print(f"\n--- Sample {task_name} ---")
        print(json.dumps(item, indent=2))

if dataset:
    print_sample("graph", dataset)
    print_sample("infobench", dataset)
    print_sample("mmlu_med", dataset)

    # save dataset
    out_path = "combined_dataset.jsonl"
    with open(out_path, "w", encoding="utf-8") as f:
        for item in dataset:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")
    print(f"\nWrote {len(dataset)} items to {out_path}")

Loading datasets...
Processing 2 examples from Graph...
Processing 2 examples from Infobench...
Processing 2 examples from MMLU Med...

--- Sample graph ---
{
  "index": 1,
  "task": "graph",
  "prompt": "You are given a directed graph with 10 nodes (numbered 0 to 9) and the following edges:\n\nEdges (source -> target, weight):\n0 -> 1, weight: 51\n0 -> 2, weight: 72\n0 -> 3, weight: 97\n0 -> 4, weight: 37\n0 -> 5, weight: 32\n0 -> 6, weight: 107\n0 -> 7, weight: 17\n0 -> 8, weight: 3\n1 -> 2, weight: 96\n1 -> 3, weight: 18\n1 -> 4, weight: 16\n1 -> 5, weight: 82\n1 -> 6, weight: 66\n1 -> 7, weight: 1\n1 -> 8, weight: 45\n1 -> 9, weight: 39\n2 -> 3, weight: 35\n2 -> 4, weight: 79\n2 -> 5, weight: 98\n2 -> 6, weight: 71\n2 -> 7, weight: 64\n2 -> 8, weight: 3\n2 -> 9, weight: 55\n2 -> 0, weight: 38\n3 -> 4, weight: 60\n3 -> 5, weight: 77\n3 -> 6, weight: 96\n3 -> 7, weight: 9\n3 -> 8, weight: 52\n3 -> 9, weight: 72\n3 -> 0, weight: 3\n3 -> 1, weight: 6\n4 -> 5, weight: 57\n4 -> 6, weight